In [ ]:
# Imports

import pandas as pd
import itertools
import random
import numpy as np
import difflib
from functools import total_ordering

def random_product(*args, repeat=1):
    "Random selection from itertools.product(*args, **kwds)"
    pools = [tuple(pool) for pool in args] * repeat
    return tuple(map(np.random.choice, pools))


## SEED BOA 1 1 2

In [ ]:
# Setup and data loading

data = pd.read_csv("data.csv").drop(columns=["Date","1st Participant Phone Number","2nd Participant Phone Number"])

appetizers_df = data.loc[data["Dish"].str.contains("Appetizer")]
main_df = data.loc[data["Dish"].str.contains("Main dish")]
dessert_df = data.loc[data["Dish"].str.contains("Dessert")]

appetizers_set = set(appetizers_df.index)
main_set = set(main_df.index)
dessert_set = set(dessert_df.index)

#events = itertools.product(appetizers,main_dish,dessert)
used_groups = []

In [ ]:
class dupla:
    def __init__(self, row) -> None:
        self.nat1 = row['P1 Nationality']
        self.nat2 = row['P2 Nationality']
        self.ind = row.name
        self.coloc = row['Coloc']
        self.restri = row['Restrictions']

    def __str__(self):
        return self.nat1 + '_' + self.nat2

def create_couples(liste):
    result = []

    for i, item1 in enumerate(liste):
        for j, item2 in enumerate(liste):
            if i != j :
                result.append([item1, item2])
    return result


a = []
for i in appetizers_df.iterrows():
    a.append(dupla(i[1]))
appetizers_set = set(a)

a = []
for i in appetizers_df.iterrows():
    a.append(dupla(i[1]))
main_set = set(a)


a = []
for i in appetizers_df.iterrows():
    a.append(dupla(i[1]))
dessert_set = set(a)

@total_ordering
class event:
    def __init__(self, name, list_of_couples) -> None:
        self.name = name
        self.list_of_couples = list_of_couples

        self.score = self.calc_score()

    def calc_score(self):
        nats = []
        for i in self.list_of_couples:
            nats.append(i.nat1)
            nats.append(i.nat2)

        nats_tuples = create_couples(nats)
        
        international_score = 1
        for i in nats_tuples:
            score = difflib.SequenceMatcher(None, i[0], i[1]).ratio()
            if score > 0.9:
                international_score += 1
        international_score = 1/international_score
        return international_score

    def __str__(self):
        s = ''
        for i in self.list_of_couples:
            s += str(i) + '_'
        return s

    def __gt__(self, other):
        return self.score > other.score

    def __eq__(self, other):
        return self.score == other.score



        




In [ ]:
#first event dessert

def create_events(name, set1, set2, set3):
    new_set = set.union(set2, set3)

    couples = create_couples(new_set)

    possible_events = []
    for i in set1:
        for j in couples:
            possible_events.append(event(name, [i] + j))

    return possible_events 

a = create_events('test', appetizers_set, main_set, dessert_set)
a.sort()
a.reverse()

for i in a:
    print(i)
        



In [ ]:
create_couples(appetizers_set)
print(appetizers_set)

In [ ]:
# Setup and data loading

data = pd.read_csv("data.csv").drop(columns=["Date","1st Participant Phone Number","2nd Participant Phone Number"])

appetizers_df = data.loc[data["Dish"].str.contains("Appetizer")]
main_df = data.loc[data["Dish"].str.contains("Main dish")]
dessert_df = data.loc[data["Dish"].str.contains("Dessert")]

appetizers_set = set(appetizers_df.index)
main_set = set(main_df.index)
dessert_set = set(dessert_df.index)

#events = itertools.product(appetizers,main_dish,dessert)
used_groups = []

In [ ]:
# Calculate appetizer groups
np.random.seed(1)

first_event = []
used_main = set()
used_dessert = set()
for pair in appetizers_set:
    group = random_product({pair},main_set-used_main,dessert_set-used_dessert)
    first_event.append(group)
    #used_groups.append(group)
    used_groups = used_groups + list(itertools.permutations(group))
    used_main.add(group[1])
    used_dessert.add(group[2])

    
appetizer_groups = []
for group in first_event:
    appetizer_groups.append(appetizers_df.loc[[group[0]]].append(main_df.loc[[group[1]]]).append(dessert_df.loc[[group[2]]]))
    

for g in appetizer_groups:
    display(g)

In [ ]:
# Calculate main_dish groups
np.random.seed(1)

second_event = []
used_appetizers = set()
used_dessert = set()
for pair in main_set:
    done = False
    while not done:
        group = random_product(appetizers_set-used_appetizers,{pair},dessert_set-used_dessert)
        if group not in used_groups:
            done = True
    second_event.append(group)
    #used_groups.append(group)
    used_groups = used_groups + list(itertools.permutations(group))
    used_appetizers.add(group[0])
    used_dessert.add(group[2])


main_groups = []
for group in second_event:
    main_groups.append(appetizers_df.loc[[group[0]]].append(main_df.loc[[group[1]]]).append(dessert_df.loc[[group[2]]]))
    
for g in main_groups:
    display(g)

In [ ]:
# Calculate dessert groups
np.random.seed(2)

third_event = []
used_appetizers = set()
used_main = set()
for pair in dessert_set:
    done = False
    while not done:
        group = random_product(appetizers_set-used_appetizers,main_set-used_main,{pair})
        if group not in used_groups:
            done = True
    third_event.append(group)
    #used_groups.append(group)
    used_groups = used_groups + list(itertools.permutations(group))
    used_appetizers.add(group[0])
    used_main.add(group[1])

dessert_groups = []
for group in third_event:
    dessert_groups.append(appetizers_df.loc[[group[0]]].append(main_df.loc[[group[1]]]).append(dessert_df.loc[[group[2]]]))
    
for g in dessert_groups:
    display(g)

In [ ]:
# Calculate people's route

address_map = {}
for person in data[" P1 Name"].append(data["P2 Name"]):
    for group in appetizer_groups:
        if group[" P1 Name"].append(group["P2 Name"]).str.contains(person).sum():
            appetizer_place = group["Coloc"].iloc[0]

    for group in main_groups:
        if group[" P1 Name"].append(group["P2 Name"]).str.contains(person).sum():
            main_place = group["Coloc"].iloc[1]

    for group in main_groups:
        if group[" P1 Name"].append(group["P2 Name"]).str.contains(person).sum():
            dessert_place = group["Coloc"].iloc[2]
    
    address_map[person] = (appetizer_place, main_place, dessert_place)

for p in address_map:
    print(p,address_map[p])
